In [169]:
import sys, os
import importlib
import numpy as np
sys.path.insert(0, '../src/')

In [170]:
special_tokens = {
    'conversation_start': '[CSTART]',
    'conversation_end': '[CEND]',
    'message_start': '[MSTART]',
    'breakpoint': '[BREAK]',
    'message_end': '[MEND]',
    'writes': '[WRITES]',
    'avoid': '[AVOID]',
}

In [171]:
import dataset
importlib.reload(dataset)

s = dataset.load_simulated_conversations('../data/simulated_conversations')

In [172]:
import dataset
importlib.reload(dataset)

m = dataset.load_messenger_conversations('/home/patz/Heap/messages/')
m = [dataset.rename_conversation_user(c, 'Patztablook TwentyTwo', 'you') for c in m]
len(m)

79

In [173]:
importlib.reload(dataset)

url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
m1 = [dataset.mark_avoid(c, url_pattern, special_tokens) for c in m]
s1 = [dataset.mark_avoid(c, url_pattern, special_tokens) for c in s]

In [174]:
importlib.reload(dataset)

def message_filter(content):
    if content is None: return True
    content = content.strip()
    blacklist = ['ə', 'ɥ', 'ʇ', 'ʎ', 'ɹ', 'ƃ',
                'se svymi geny? 0=ne', 'are you satisfied with ur genes?', 'would you describe urself as',
                'nyom']
    for b in blacklist:
        if b in content: return False
    if content in ['.', '*'] : return False # probably a reference to a previous message
    # fixing previous a previous message (probably a typo):
    if content[0] == '*' and content[1] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False
    if content[-1] == '*' and content[-2] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False
    return True

m2 = [dataset.filter_messages_by_content(c, message_filter) for c in m1]

In [175]:
m2 = list(filter(lambda c: len(c) >= 64, m2))
print(len(m2))
for c in m:
    print(c.title, c.participants, len(c))

49
Scanthning Brot puɐ Vincent Fournier {'you', 'Scanthning Brot', 'Vincent Fournier'} 7
Pendulum Swing {'Sigismund Häggkvist', 'you', 'Robin Stringer', 'Martin McNickle', 'Rawen Dorfin'} 15750
Amélie Novotná {'you', 'Amélie Novotná'} 156
Martin McNickle {'you', 'Martin McNickle'} 6
Ondřej Smolík {'you', 'Ondřej Smolík'} 176
Alexander Terziev {'you', 'Alexander Terziev'} 4433
Manka Roblíková {'you', 'Manka Roblíková'} 745
Matouš Pikous {'you', 'Matouš Pikous'} 35
Jan Zasadil {'Jan Zasadil', 'you'} 38947
Katie Alzbeta Brown {'you', 'Katie Alzbeta Brown'} 102
Jan Vaniš {'Jan Vaniš', 'you'} 407
Katie Alzbeta Brown puɐ Karel Loula {'Karel Loula', 'you', 'Katie Alzbeta Brown'} 183
Filip Kastl {'you', 'Filip Kastl'} 6845
The only magician capable of a craft left his stove on {'you', 'Scanthning Brot'} 407
RJ {'Jakub Tichanek', 'Alexander Terziev', 'you', 'Adam Dvořák', 'Adam Vanĕček', 'Petr Hošek', 'Šimon Dutý', 'Patrik Kabrna', 'Adéla Gabesamová', 'Abudi  Shalabi', 'Štěpán Chrast', 'Malcolm

In [176]:
val_indices = [] #[-1, -3, -13]
train_indices = np.delete(np.arange(len(m2)), val_indices)

print(len(m2))
for i in val_indices:
    print(m2[i].participants)

49


In [179]:
import dataset
importlib.reload(dataset)

train_conversations = [m2[i] for i in train_indices] + s1
val_conversations = [m2[i] for i in val_indices]

dataset.generate_corpus('../data/train.txt', train_conversations, special_tokens, break_tollerance_s=180)
dataset.generate_corpus('../data/val.txt', val_conversations, special_tokens, break_tollerance_s=180)